## **1. Mount google drive**
---

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


## **2. Import the libraries**
---

In [2]:
import pickle
import cv2
import sys
import os
import matplotlib
import sklearn
import numpy as np
import tensorflow as tf

from imutils import paths
from collections import deque
from tensorflow.keras.models import load_model
from IPython.display import clear_output

print("Versions of key libraries")
print("---")
print("tensorflow:", tf.__version__)
print("numpy:     ", np.__version__)
print("opencv    :", cv2.__version__)

Versions of key libraries
---
tensorflow: 2.4.1
numpy:      1.19.5
opencv    : 4.1.2


## **3. Setup the size for deque object (`qsize`) and various paths**
---
* `modelpath`: Path to the trained model
* `pickpath` : Path to the pickle file that stores the labelBinarizer object
* `videopath`: Path to the video of interest
* `outpath`  : Path to store the output of video analysis

In [3]:
qsize       = 1

datapath    = "/content/gdrive/My Drive/iss/RTAVS/Week 3/data/"
colabpath   = "/content/gdrive/My Drive/iss/RTAVS/Week4/colab/"

modelName   = 'sportsV1'
videoName   = 'awvolleyball.mp4'
outName     = videoName[:-4]+'_'+str(qsize)+'.avi'

modelpath   = os.path.join(datapath,modelName+".hdf5")
pickpath    = os.path.join(datapath,modelName+".pickle")
videopath   = os.path.join(datapath,videoName)
outpath     = os.path.join(colabpath,outName)

print("Path to model :",modelpath)
print("Path to pickle:",pickpath)
print("Path to video :",videopath)
print("Path to output:",outpath)

Path to model : /content/gdrive/My Drive/iss/RTAVS/Week 3/data/sportsV1.hdf5
Path to pickle: /content/gdrive/My Drive/iss/RTAVS/Week 3/data/sportsV1.pickle
Path to video : /content/gdrive/My Drive/iss/RTAVS/Week 3/data/awvolleyball.mp4
Path to output: /content/gdrive/My Drive/iss/RTAVS/Week4/colab/awvolleyball_1.avi


## **4. Load the trained model**
---

In [4]:
model     = load_model(modelpath)
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
______________________________________________________________________________________________

## **5. Before the analysis**
---
* Step 1: Setup the mean to be subtracted from each frame
* Step 2: Setup deque object
* Step 3: Load the pre-saved labelBinarizer object from pickle file
* Step 4: Load the video. Initialize `writer`. Set width and height of a frame to `None`.

In [5]:
imgMean = np.array([123.68,116.779,103.939],                                    # Step 1
                   dtype="float32")
                                                                                
Q       = deque(maxlen=qsize)                                                   # Step 2

lb      = pickle.loads(open(pickpath, "rb").read())                             # Step 3

                                                                                # Step 4
vs      = cv2.VideoCapture(videopath)
writer  = None
(W, H)  = (None, None)

print("The shape of Q:", Q)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.preprocessing.label module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.preprocessing. Anything that cannot be imported from sklearn.preprocessing is now part of the private API.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator LabelBinarizer from version 0.21.3 when using version 0.22.2.post1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


The shape of Q: deque([], maxlen=1)


## **6. Run the analysis (to be completed)**
---
* Step 1: Setup running number `fr` for reporting of the frame being analyzed
* Step 2: Read a frame from video stream
* Step 3: If there is no frame left to be analyzed, exit the while loop
* Step 4: Report the frame being analyzed
* Step 5: If `W` and `H` are still `None`, extract the value from the shape of a frame
* Step 6: Duplicate the frame of interest. Convert the frame to RGB. Resize the frame. Subtract the frame from the specified mean
* Step 7: Perform prediction on the frame. Add the result to `Q`, the deque object.
* Step 8: Perform a mean on `Q`. Get the prediction of class for this frame
* Step 9: Generate the text and put it on the frame
* Step 10: Setup the writer if it is yet to be setup
* Step 11: Write the frame into the video stream
* Step 12: After all frames are done, close the writer and release video stream (of the original video)

In [6]:
fr      = 1                                                                     # Step 1

while True:
                                                                                # Step 2
    (grabbed, frame) = vs.read()
                                                                                # Step 3
    if not grabbed:
        break
                                                                                # Step 4
                                                                                # Step 5
    if W is None or H is None:
        (H, W) = frame.shape[:2]
                                                                                # Step 6
    output = frame.copy()
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    frame = cv2.resize(frame, (224, 224)).astype("float32")
    frame -= imgMean
                                                                                # Step 7
    preds = model.predict(np.expand_dims(frame,axis=0))[0]
    Q.append(preds)
                                                                                # Step 8
    predout = np.array(Q).mean(axis=0)
    clss = np.argmax(predout)
    label = lb.classes_[clss]
                                                                                # Step 9
    text = "Event: {}".format(label)
    cv2.putText(output,
                text,
                (10,40),
                cv2.FONT_HERSHEY_SIMPLEX,
                1.25,
                (0,255,0),
                5,
                cv2.LINE_AA)
                                                                                # Step 10
    if writer is None:
        fourcc = cv2.VideoWriter_fourcc(*"MJPG")
        writer = cv2.VideoWriter(outpath,
                                 fourcc,
                                 30,
                                 (W, H),
                                 True)

                                                                                # Step 11
    writer.write(output)

                          # Real-time display the output image
#    cv2.imshow("Output", output)

                                                                                # Step 12
print("Closing ...")
writer.release()
vs.release()

Closing ...
